In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.spatial import procrustes

data_path = '/Users/jamesburgess/research/robust-dim-reduction-popgen'

# Read sample label & country lookup data

In [ ]:
popres_samples = pd.read_csv('{}/data/labels.csv'.format(data_path)).set_index('indID')
popres_samples.groupby('label').count()

In [ ]:
country_lookup = pd.read_csv('{}/data/country-origins.csv'.format(data_path), sep=",")\
                    .set_index('Geographic-Origin')
country_lookup

# Functions

In [ ]:
"""
For datasets where there is one row per label/sample, indexed by 'indID'. 
Take a centroid of the data points as the predicted country location
This will not correctly plot data that is already country-averaged
"""
def run_plot(PC_projection, labels, show_plot=True, savefig=True):
    assert len(PC_projection) == len(labels)

    # find the countries corresponding to each label, build new dataframe
    countries = labels.join(popres_samples, rsuffix='_')['label']
    df = pd.DataFrame(data=PC_projection, index=countries)
    
    # find sample means  
    df_countries = df.groupby(df.index).mean()
    n=len(df_countries)
    print("\nData has {} distinct countries/groups".format(n))
    
    # Do procrustes rotation to align the map
    d2 = df_countries[[0,1]]  # top 2 PCs
    d1 = country_lookup.loc[d2.index][['Longitude', 'Latitude']] 
    
    d1_normalized, d1_rotated, disparity = procrustes(d1, d2)
    print("Doing Procrustes transform. $M^2$ error = {:.4f}".format(disparity))
    
    # plotting and saving
    if show_plot:
        f, ax = plt.subplots(1,1, figsize=(20,20))
        for i in range(len(d1_rotated)):
            country = d1.iloc[i].name
            country_abbreviation = country_lookup.loc[country]['Abbreviation']
            ax.scatter(d1_rotated[i,0], d1_rotated[i,1]
                            , marker="${}$".format(country_abbreviation)
                            , s=1000)
        if savefig:
            fname = '{}/results/{}-country-mean-projection.png'.format(data_path, fname_prefix)
    return disparity, n

def run_plot_from_label_id(fname_prefix, show_plot=True, savefig=True):
    # Read and report data 
    fname_PC_projection = '{}/results/{}-PC_projection.dat'.format(data_path, fname_prefix)
    fname_labels = '{}/results/{}-labels.csv'.format(data_path, fname_prefix)
    print("Reading:\n\t{}\n\t{}".format(fname_PC_projection, fname_labels))
    PC_projection = np.loadtxt(fname_PC_projection)
    labels = pd.read_csv(fname_labels).set_index('indID')
    assert len(PC_projection) == len(labels)
    print("Read {} data poitns".format(len(labels)))
    
    run_plot(PC_projection, labels, show_plot=True, savefig=True)

In [ ]:
''' 
Test variyng the `disparity` metric that's output by scipy.spatial.procrustes
Look at whether normalizing is appropriate. 
'''
def run_disparit_metric_experiment():
    raise NotImplemented()

# Projections: Vanilla PCA

In [ ]:
fname_prefix = 'pca-countries_gt_4_n_samples-removed_outliers-filter_20_samples_p_cntry'
_ = run_plot_by_label_id(fname_prefix)    

In [ ]:
fname_prefix = 'pca-countries_gt_4_n_samples-removed_outliers-no_other_filtering'
_ = run_plot_by_label_id(fname_prefix)    

# Projections: Normalized PCA

2 Cases
- First using similarity metric $\frac{1}{d^1}$ for the Laplacian
- Second using similarity metric $\frac{1}{d^2}$ for the Laplacian

In [ ]:
fname_prefix = 'norm-countries_gt_4_n_samples-pca-no_removed_outliers-no_other_filters-norm_pca-inv_pow_1'
_ = run_plot_by_label_id(fname_prefix)    

In [ ]:
fname_prefix = 'norm-countries_gt_4_n_samples-pca-no_removed_outliers-no_other_filters-norm_pca-inv_pow_2'
_ = run_plot_by_label_id(fname_prefix)    

# Supervised PCA 
The same 2 cases as normalized PCA. 

Sets $t=0$.

In [ ]:
fname_prefix = 'supervised_pca_t_0-countries_gt_4_n_samples-pca-no_removed_outliers-no_other_filters-norm_pca-inv_pow_1'
_ = run_plot_by_label_id(fname_prefix)    

In [ ]:
fname_prefix = 'supervised_pca_t_0-countries_gt_4_n_samples-pca-no_removed_outliers-no_other_filters-norm_pca-inv_pow_2'
_ = run_plot_by_label_id(fname_prefix)    